In [1]:
import torch
from ksmm_triton.ksmm_module import create_butterfly_patterns, create_butterfly_chain


Generated 10 configurations


In [2]:
orig_weight = torch.load("ckpt/gate_proj_weights0.pth")

In [3]:
orig_weight.shape

torch.Size([2048, 720])

In [19]:
shape = orig_weight.shape
chain = create_butterfly_chain(shape, rank=4, dtype=torch.float16, device='cuda', bs_last=False)

In [20]:
batch_size = 32
x = torch.randn( batch_size, shape[1], dtype=torch.float16, device='cuda')
output = chain(x)  # Shape: (batch_size, shape[0])

In [23]:
output.shape

torch.Size([32, 2048])

In [18]:
chain

KSLinearTriton(
  in_features=720, out_features=2048, bias=True, patterns=[(1, 4, 720, 1), (1, 2048, 4, 1), (1024, 2, 2, 1), (512, 2, 2, 2), (256, 2, 2, 4), (128, 2, 2, 8), (64, 2, 2, 16), (32, 2, 2, 32), (16, 2, 2, 64), (8, 2, 2, 128), (4, 2, 2, 256), (2, 2, 2, 512), (1, 2, 2, 1024)], dtype=torch.float16, bs_last=True
  (weights): ParameterList(
      (0): Parameter containing: [torch.float16 of size 1x720x4 (cuda:0)]
      (1): Parameter containing: [torch.float16 of size 1x4x2048 (cuda:0)]
      (2): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (3): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (4): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (5): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (6): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (7): Parameter containing: [torch.float16 of size 1024x2x2 (cuda:0)]
      (8): Parameter containing: [torch.float16 of s

In [8]:
patterns = chain.patterns

In [9]:
total_params = sum(a * b * c * d for a, b, c, d in patterns)
dense_params = orig_weight.numel()

print(f"Total parameters in chain: {total_params}")
print(f"Total parameters in dense weight: {dense_params}")
print(f"Compression ratio: {dense_params / total_params:.2f}")

Total parameters in chain: 56128
Total parameters in dense weight: 1474560
Compression ratio: 26.27


In [10]:
dense_params

1474560

In [11]:
720 * 2048

1474560

In [12]:
orig_weight.shape

torch.Size([2048, 720])

In [13]:
patterns

[(1, 4, 720, 1),
 (1, 2048, 4, 1),
 (1024, 2, 2, 1),
 (512, 2, 2, 2),
 (256, 2, 2, 4),
 (128, 2, 2, 8),
 (64, 2, 2, 16),
 (32, 2, 2, 32),
 (16, 2, 2, 64),
 (8, 2, 2, 128),
 (4, 2, 2, 256),
 (2, 2, 2, 512),
 (1, 2, 2, 1024)]